In [28]:
from pyspark.sql import SparkSession

In [29]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

23/09/12 20:05:29 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [32]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [33]:
my_log_reg = LogisticRegression()

In [34]:
fitted_logreg = my_log_reg.fit(my_data)

In [35]:
log_summary = fitted_logreg.summary

In [36]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [37]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [38]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [39]:
final_model = LogisticRegression()

In [40]:
fit_final = final_model.fit(lr_train)

In [41]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [42]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[43.1251747371069...|           [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,148...|[42.1353900213067...|           [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[34.4099608650169...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[36.0633744321202...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[24.0647320903520...|[0.99999999996461...|       0.0|
|  0.0|(692,[125,126,127...|[33.5892852765644...|[0.99999999999999...|       0.0|
|  0.0|(692,[127,128,129...|[16.9556081328659...|[0.99999995672142...|       0.0|
|  0.0|(692,[127,128,129...|[27.8647027551284...|[0.99999999999920...|       0.0|
|  0.0|(692,[129,130,131...|[17.4584431391492...|[0.99999997382453...|       0.0|
|  0.0|(692,[152

In [43]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [44]:
my_eval = BinaryClassificationEvaluator()

In [45]:
final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [46]:
final_roc

0.9829059829059829